Empezamos importando librerias

In [67]:
import numpy as np
import scipy as sp

In [68]:
def descompLU_recursive(A):
    #get dimensions of A
    n = A.shape[0]

    u_11 = A[0,0]
    if (u_11 == 0): # whe cannot pivot anymore, the A matrix cannot be descomposed
        return (None, None)

    if (n == 1):
        return (np.array([[1]]), np.array([[u_11]]))
    

    u_12 = A[0, 1:]

    l_21 = A[1:, 0] / u_11

    recursive_A_matrix_to_solve = A[1:, 1:] - l_21.reshape(n - 1,1) * u_12

    l_22, u_22 = descompLU_recursive(recursive_A_matrix_to_solve)
    if (l_22 is None or u_22 is None): # returns canonical and original A
        return (None, None)
    

    l_top_row = np.array([1] + [0] * (n - 1))
    #create a new matrix using the l_22 matrix with dimensions of (n - 1, n - 1) and the l_21 matrix with dimensions of (n - 1, 1)
    l = np.hstack((np.expand_dims(l_21, axis=1), l_22))
    l = np.vstack((l_top_row, l))

    #create a new matrix using the l matrix with dimensions of (n - 1, n) and the l_top_row matrix with dimensions of (1, n)
    u_left_row = np.array([u_11] + [0] * (n - 1))
    #create a new matrix using the u_22 matrix with dimensions of (n - 1, n - 1) and the u_12 matrix with dimensions of (1, n - 1)
    u = np.vstack((u_12, u_22))
    #create a new matrix using the u matrix with dimensions of (n, n - 1) and the u_left_row matrix with dimensions of (n, 1)
    u = np.hstack((np.expand_dims(u_left_row, axis=1), u))

    return (l, u)

def descompLU(A):
    l, u = descompLU_recursive(A)
    if (l is None or u is None): # returns canonical and original A
        return (np.eye(A.shape[0]), A)
    return (l, u)


In [69]:
A0 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
A1 = np.array([[ 4,3],[-5,9]])
A2 = np.array([[4, 3, 2], [-2, 2, 3], [3, -5, 2]])
A3 = np.array([[20, 10], [17, 22]])
A4 = np.array([[8, 3, -2], [-4, 7, 5], [3, 4, -12]])
A5 = np.array([[4, 3, -5], [-2, -4, 5], [8, 8, 0]])

def test_descompLU(test_suite):
    for test_case in test_suite:
        L1,U1 = descompLU(test_case)
        if (np.allclose(test_case, L1@U1)):
            print("OK")
        else:
            print("ERROR")
            print(L1)
            print(U1)

A_test_suite = [A0,A1,A2,A3,A4,A5]
test_descompLU(A_test_suite)

OK
OK
OK
OK
OK
OK


In [70]:
def resolverLU(A, b):
    l, u = descompLU(A)
    x = sp.linalg.solve_triangular(l, b, lower=True)
    return sp.linalg.solve_triangular(u, x)

In [71]:
A = np.array([[ 4,3],[-5,9]])
b = np.array([20,26])
A2 = np.array([[4, 3, 2], [-2, 2, 3], [3, -5, 2]])
b2 = np.array([25, -10, -4])
A3 = np.array([[20, 10], [17, 22]])
b3 = np.array([350, 500])
A4 = np.array([[8, 3, -2], [-4, 7, 5], [3, 4, -12]])
b4 = np.array([9, 15, 35])
A5 = np.array([[4, 3, -5], [-2, -4, 5], [8, 8, 0]])
b5 = np.array([2, 5, -3])

def testresolverLU(A_test_suite, b_test_suite):
    for (A_test_case, b_test_case) in zip(A_test_suite, b_test_suite):
        solucion = resolverLU(A_test_case,b_test_case)
        solucion_posta = np.linalg.solve(A_test_case,b_test_case)
        if (np.allclose(solucion, solucion_posta)):
            print("OK")
        else:
            print("ERROR")
            print(solucion)
            print(solucion_posta)

A_test_suite = [A,A2,A3,A4,A5]
b_test_suite = [b,b2,b3,b4,b5]
testresolverLU(A_test_suite,b_test_suite)

OK
OK
OK
OK
OK


In [72]:
def inversa(A):
    dimension = A.shape[0]
    canonical_matrix = np.eye(dimension)
    ret = np.zeros((dimension, dimension))

    for i in range(dimension):
        inverse_canonical_row = resolverLU(A, canonical_matrix[i])
        ret[:, i] = inverse_canonical_row

    return ret

In [73]:
A0 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 10]])
A1 = np.array([[ 4,3],[-5,9]])
A2 = np.array([[4, 3, 2], [-2, 2, 3], [3, -5, 2]])
A3 = np.array([[20, 10], [17, 22]])
A4 = np.array([[8, 3, -2], [-4, 7, 5], [3, 4, -12]])
A5 = np.array([[4, 3, -5], [-2, -4, 5], [8, 8, 0]])

def testinversa(test_suite):
    for test_case in test_suite:
        A_inversa = inversa(test_case)
        A_inversa_posta = np.linalg.inv(test_case)
        if (np.allclose(A_inversa_posta, A_inversa)):
            print("OK")
        else:
            print("ERROR")
            print (A_inversa)
            print (A_inversa_posta)

A_test_suite = [A0,A1,A2,A3,A4,A5]
testinversa(A_test_suite)

OK
OK
OK
OK
OK
OK
